given a library of midi loops, meander through it, always playing the most similar unplayed loop


## functions


In [1]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    bar_length = 100
    # decimals = 1
    # percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    percent = f"{iteration:.2f}/{total:.2f}"
    filledLength = int(bar_length * iteration // total)
    bar = fill * filledLength + '-' * (bar_length - filledLength)
    print(f'\r{prefix} |{bar}| {percent} {suffix}', end = printEnd)

    if iteration == total: 
        print()

In [2]:
import mido
import pretty_midi

def normalize_midi(mid: pretty_midi.PrettyMIDI):
  """Normalize MIDI file so that the first note is at 0.0s."""
  for instrument in mid.instruments:
    if not instrument.notes:
      continue
    first_note_start = instrument.notes[0].start
    for note in instrument.notes:
      note.start -= first_note_start
      note.end -= first_note_start
  return mid

In [21]:
from scipy.spatial.distance import cosine

def find_most_similar_vector(target_vector, vector_array):
  most_similar_vector = None
  highest_similarity = -1  # since cosine similarity ranges from -1 to 1

  for vector_data in vector_array:
    name, vector = vector_data.values()
    similarity = 1 - cosine(target_vector, vector)  # Cosine similarity
    if similarity > highest_similarity:
      highest_similarity = similarity
      most_similar_vector = name

  return most_similar_vector

## calculate metrics


In [4]:
import os
import chunk_utils as cu

midi_dict = {}
folder_path = os.path.join("data", "outputs", "MIDI1")
config = {"w1": 0.5, "w2": 0.5, "bin_length": 1.0}

for file in os.listdir(folder_path):
  if file.endswith('.mid') or file.endswith('.midi'):
    file_path = os.path.join(folder_path, file)
    midi = pretty_midi.PrettyMIDI(file_path)

    normalized_midi = normalize_midi(midi)
    metrics = cu.all_metrics(midi, config)

    midi_dict[file] = {
      "notes": normalized_midi,
      "metrics": metrics,
      "played": False,
    }

## play


In [22]:
import numpy as np

# choose random initial file
loop_chance = 0.5 # pct
play_time = 60 # seconds
time_elapsed = 0 # seconds
midi_device_name = "to Max 1"
seed = np.random.randint(0, len(midi_dict.items()) - 1)
current_file_path = list(midi_dict.items())[seed][0]
next_file_path = None
port = mido.open_output(midi_device_name) # type: ignore

while (time_elapsed < play_time):
  # find next file
  if (np.random.rand() < loop_chance):
    cph = list(midi_dict.items())[seed][1]['metrics']['pitch_histogram']
    all_vecs = [
      {'name': filename, 'metric': details['metrics']['pitch_histogram']}
      for filename, details in midi_dict.items()
      if not details['played']
    ]
    next_file_path = find_most_similar_vector(cph, all_vecs)
  else:
    next_file_path = current_file_path
  print(f"[{time_elapsed:2.2f}/{play_time:2.2f}]\tPlaying {current_file_path} (next up is {next_file_path})")

  # play file
  t = 0
  current_file = mido.MidiFile(os.path.join(folder_path, current_file_path)) # type: ignore
  for msg in current_file.play():
    printProgressBar(t, current_file.length, suffix='s')
    t += msg.time # type: ignore
    port.send(msg)
  printProgressBar(current_file.length, current_file.length, suffix='s')
  time_elapsed += current_file.length

  # ready next file
  midi_dict[current_file_path]['played'] = True
  current_file_path = next_file_path


[0.00/60.00]	Playing MIDI1-12.mid (next up is MIDI1-12.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 6.37/6.37 s
[6.37/60.00]	Playing MIDI1-12.mid (next up is MIDI1-12.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 6.37/6.37 s
[12.73/60.00]	Playing MIDI1-12.mid (next up is MIDI1-10.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 6.37/6.37 s
[19.10/60.00]	Playing MIDI1-10.mid (next up is MIDI1-10.mid)
 |███████████████████████████████████████████████████████████████████████████████████████████████████-| 7.07/7.07 s
[26.17/60.00]	Playing MIDI1-10.mid (next up is MIDI1-10.mid)
 |███████████████████████████████████████████████████████████████████████████████████████████████████-| 7.07/7.07 s
[33.24/60.00]	Playing MIDI1-10.mid (next up is MIDI1-5.mid)
 |███████████████████████████████████████████████████████